In [1]:
import logging
import os
import socket
from dask.distributed import Client, LocalCluster, get_task_stream, progress, wait
from dask_jobqueue import LSFCluster
from time import perf_counter, sleep

In [ ]:
n_workers_per_node = 16 # number of worker processes per node
node_memory = 16 * 200 # node memory in GB
n_threads_per_worker = 1
worker_time = 120 # job time per node for worker
worker_queue = "pdebug" # queue to be used per worker
import socket
host = socket.gethostname()
dashboard_address = '{}:8788'.format(socket.gethostname())
memory = '{}GB'.format(node_memory/n_workers_per_node)
job_extra = ['-nnodes 1', 
             '-G asccasc', 
             '-q {}'.format(worker_queue), 
             '-W {}'.format(worker_time)]
local_directory="/var/tmp/dask"
cluster = LSFCluster(cores = n_workers_per_node*n_threads_per_worker,processes=n_workers_per_node, memory='{}GB'.format(node_memory), 
                     header_skip=['-n ','-R','-M', '-P', '-W 00:30'], 
                     job_extra = job_extra, 
                     death_timeout=worker_time*60,
                     local_directory =f"{local_directory}",
                     use_stdin=True, 
                     scheduler_options={"port": 0,"host":host, "dashboard_address": ':8788'})

#cluster = LocalCluster(n_workers=40,local_directory =f"{local_directory}/local", dashboard_address= f':8797')

client = Client(cluster, set_as_default=True)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.66.201:8788/status,
Dashboard: http://192.168.66.201:8788/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.66.201:45531,Workers: 0
Dashboard: http://192.168.66.201:8788/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
cluster.scale(n_workers_per_node)

/usr/workspace/iopp/projects/digio-venv-lassen/lib/python3.9/site-packages/dask_jobqueue/core.py:251: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
